# Saving Full Data to SQL

In [4]:
import pandas as pd 

full_data = pd.read_csv("cleaned_final_data_utf-8.csv")
full_data.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,...,Max_temp,Precip,Humidity,Heat_Index,Latitude_y,Longitude_y,Elevation,review_neg,review_neu,review_pos
0,1,US,"Overripe and Porty, with raisin, prune and cho...",Reserve,81,37,California,Paso Robles,Central Coast,NaN,...,100.4,7.94,65.69,101.7,33.343447,-117.203325,190.0,0.000,0.829,0.171
1,2,US,"Strong aromas of blueberry paste, cracked pepp...",Paso Bordo Reserve,92,69,California,Paso Robles,Central Coast,Matt Kettmann,...,100.4,7.94,65.69,101.7,33.343447,-117.203325,190.0,0.000,0.805,0.195
2,3,US,A vegetal note drags down the enjoyment. On th...,Reserve,84,39,California,Paso Robles,Central Coast,NaN,...,100.4,7.94,65.69,101.7,33.343447,-117.203325,190.0,0.129,0.710,0.162
3,4,US,Larry Stanton patiently waits to release the w...,La Terraza Bloc Reserve,93,59,California,Paso Robles,Central Coast,Matt Kettmann,...,100.4,7.94,65.69,101.7,33.343447,-117.203325,190.0,0.039,0.865,0.096
4,5,US,"High alcohol gives the wine heat, especially i...",La Tertazza Bloc Reserve,85,39,California,Paso Robles,Central Coast,NaN,...,100.4,7.94,65.69,101.7,33.343447,-117.203325,190.0,0.000,0.895,0.105


In [5]:
full_data = full_data.rename(columns={"Unnamed: 0": 'wine_id', "Min_temp": 'min_temp', "Max_temp": 'max_temp', "Precip": 'precip', 
                                     "Humidity": 'humidity', "Heat_Index": 'heat_index', "Latitude_x": 'latitude_x', 
                                     "Longitude_x": 'longitude_x', "Latitude_y": 'latitude_y', 
                                     "Longitude_y": 'longitude_y', "Elevation": 'elevation'})
full_data.head()

,wine_id,country,description,designation,points,price,province,region_1,region_2,taster_name,...,max_temp,precip,humidity,heat_index,latitude_y,longitude_y,elevation,review_neg,review_neu,review_pos
0,1,US,"Overripe and Porty, with raisin, prune and cho...",Reserve,81,37,California,Paso Robles,Central Coast,NaN,...,100.4,7.94,65.69,101.7,33.343447,-117.203325,190.0,0.000,0.829,0.171
1,2,US,"Strong aromas of blueberry paste, cracked pepp...",Paso Bordo Reserve,92,69,California,Paso Robles,Central Coast,Matt Kettmann,...,100.4,7.94,65.69,101.7,33.343447,-117.203325,190.0,0.000,0.805,0.195
2,3,US,A vegetal note drags down the enjoyment. On th...,Reserve,84,39,California,Paso Robles,Central Coast,NaN,...,100.4,7.94,65.69,101.7,33.343447,-117.203325,190.0,0.129,0.710,0.162
3,4,US,Larry Stanton patiently waits to release the w...,La Terraza Bloc Reserve,93,59,California,Paso Robles,Central Coast,Matt Kettmann,...,100.4,7.94,65.69,101.7,33.343447,-117.203325,190.0,0.039,0.865,0.096
4,5,US,"High alcohol gives the wine heat, especially i...",La Tertazza Bloc Reserve,85,39,California,Paso Robles,Central Coast,NaN,...,100.4,7.94,65.69,101.7,33.343447,-117.203325,190.0,0.000,0.895,0.105


In [6]:
from sqlalchemy import create_engine


protocol = 'postgresql'
username = 'postgres'
password = 'administrator'
host = 'wine.corp16oy9wyq.us-east-1.rds.amazonaws.com'
port = 5432

rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}'
        
engine = create_engine(rds_connection_string)

In [8]:
# conn = engine.connect()
# conn.execute("commit")
# conn.execute("create database wine")

In [9]:
database_name = 'wine'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)
conn = engine.connect()

In [45]:
columnName = list(full_data.columns.values)

def getColumnDtypes(dataTypes):
    dataList = []
    for x in dataTypes:
        if(x == 'int64'):
            dataList.append('int')
        elif (x == 'float64'):
            dataList.append('float')
        elif (x == 'bool'):
            dataList.append('boolean')
        else:
            dataList.append('varchar')
    return dataList

columnDataType = getColumnDtypes(full_data.dtypes)

createTableStatement = 'CREATE TABLE IF NOT EXISTS wine_data ('
for i in range(len(columnDataType)):
    createTableStatement = createTableStatement + '\n' + columnName[i] + ' ' + columnDataType[i] + ','
createTableStatement = createTableStatement[:-1] + ' );'

conn.execute("commit")
conn.execute(createTableStatement)

In [10]:
from sqlalchemy import inspect

inspector = inspect(engine)

inspector.get_table_names()

['wine_data']

In [11]:
full_data.to_sql(name='wine_data', con=conn, if_exists='append', index=False)

In [12]:
print(conn.execute("SELECT * FROM wine_data").first())


(1, 'US', 'Overripe and Porty, with raisin, prune and chocolate flavors that finish hot and slightly sweet. The heat simply overwhelmed this Merlot.', 'Reserve', 81, 37.0, 'California', 'Paso Robles', 'Central Coast', None, None, 'Cerro Prieto 2008 Reserve Merlot (Paso Robles)', 'Merlot', 'Cerro Prieto winery', 33.3434468, -117.2033251, 34.4, 100.4, 7.94, 65.69, 101.7, 33.3434468, -117.2033251, 190.0, 0.0, 0.8290000000000001, 0.171)
